## Import libraries

In [2]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import pandas as pd
import datetime as dt
import math 

In [6]:
# create global list of targeted neighbourhoods AND match with platform location IDs
neighbourhoods = ['Hammersmith and Fulham', 'Kensington and Chelsea', 'Camden', 'City of Westminster', 'City of London', 'Hackney', 'Lambeth', 'Tower of Hamlets', 'Islington']
rightmove_loc_id = ['5E61407', '5E61229', '5E93941', '5E61233', '5E61224', '5E93953', '5E93971', '5E61417', '5E93965']
zip_iterator = zip(neighbourhoods, rightmove_loc_id)
locations_dict = dict(zip_iterator)
    # {'Hammersmith and Fulham': '5E61407',
    # 'Kensington and Chelsea': '5E61229',
    # 'Camden': '5E93941',
    # 'City of Westminster': '5E61233',
    # 'City of London': '5E61224',
    # 'Hackney': '5E93953',
    # 'Lambeth': '5E93971',
    # 'Tower of Hamlets': '5E61417',
    # 'Islington': '5E93965'}
furnishTypes_lst = ['furnished', 'unfurnished', 'partFurnished' ]   # partFurnished gives back more than delta

### page count function

In [153]:
# returns list of page indices we can use for the url to check every single page of search results
def page_count(bs):
    result_count = int(bs.find(class_="searchHeader-resultCount").get_text())
    num_pages = math.ceil(result_count / 24) # don't know why, but have to use 24 instead of 25 (results per page)
    page_indices = [x*24 for x in range(0, num_pages)]
    return(page_indices)

page_indices = page_count(bs)

### ID-Function

In [ ]:
# get internal IDs
def get_ids_rm(bs):
    ids = bs.find_all(class_ = 'l-searchResult is-list')
    ids_lst = [id.get('id') for id in ids]
    ids_lst = [(str(x)) for x in ids_lst]
    return(ids_lst)

get_ids_rm(bs)
# ids_lst

### Price functions


In [179]:
# function for prices monthly and weekly(=sec_prices)

def get_prices_rm(bs):
    prices = bs.find_all(class_= "propertyCard-priceValue")
    prices_lst = [price.get_text() for price in prices]
    # format prices
    prices_lst = [price.removesuffix(' pcm').replace('£', '').replace(',', '') for price in prices_lst]
    #prices_lst = [float(price) for price in prices_lst]
    prices_lst = list(filter(None, prices_lst))
    return(prices_lst)  


def get_sec_prices_rm(bs):
    sec_prices = bs.find_all(class_= "propertyCard-secondaryPriceValue")
    sec_prices_lst = [sec_price.get_text() for sec_price in sec_prices]
    #format sec_prices
    sec_prices_lst = [sec_price.removesuffix(' pw').replace('£', '').replace(',', '') for sec_price in sec_prices_lst]
    #sec_prices_lst = [float(sec_price) for sec_price in sec_prices_lst]
    sec_prices_lst = list(filter(None, sec_prices_lst))
    return(sec_prices_lst)

In [ ]:
list2 = [1, 6, [], 3, [], [], 9]

res = list(filter(None, list2))
print(res)

In [176]:
get_prices_rm(bs)

['3748', '3120', '2383']

In [184]:
# # automated scraping of different neighbourhoods
# url = f"https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%{rightmove_loc_id[0]}&index={page_indices[0]}&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=houseShare%2Cretirement%2Cstudent&furnishTypes={furnishTypes_lst[0]}&keywords="
# page = requests.get(url)
# html = page.content


In [ ]:
https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%5E61407&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=houseShare%2Cretirement%2Cstudent&furnishTypes=furnished&keywords=

In [7]:
b# Static URL:
# Neighbourhood 1 - Hammersmith and Fulham; furnished
page = requests.get("https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%5E61407&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=houseShare%2Cretirement%2Cstudent&furnishTypes=furnished&keywords=")
html = page.content
bs = BeautifulSoup(html, 'html.parser')


In [172]:
# Static URL:
# Neighbourhood 1 - Hammersmith and Fulham; furnished
page = requests.get("https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%5E61407&index=480&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=houseShare%2Cretirement%2Cstudent&furnishTypes=furnished&keywords=")
html = page.content
bs = BeautifulSoup(html, 'html.parser')

In [185]:
page_indices = page_count(bs)

In [ ]:
page_indices

In [188]:
df_search_rm = pd.DataFrame() 

In [138]:
x = '5E61407'


In [189]:
for p in page_indices:
    
    page = requests.get(f"https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%{x}&index={p}&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=houseShare%2Cretirement%2Cstudent&furnishTypes={furnishTypes_lst[0]}&keywords=")
    html = page.content
    bs = BeautifulSoup(html, 'html.parser')

    rightmove_dict = {
    'id': get_ids_rm(bs),
    #'location': locations_dict[location],
    'prices_pcm': get_prices_rm(bs),
    'prices_pw': get_sec_prices_rm(bs)
    }

    df_page_rm = pd.DataFrame(data=rightmove_dict) # dataframe with 24 rows of objects (=1 page of results)
    df_page_rm['location'] = list(locations_dict.keys())[0]
    df_page_rm['furnish_type'] = furnishTypes_lst[0]

    # append obove dataframe to main df
    df_search_rm = pd.concat([df_search_rm, df_page_rm], axis=0, ignore_index=True)
    # df_search_rm = df_search_rm.drop_duplicates()

    # if dataframe needs to be deleted:
    # df_search_rm = df_search_rm.iloc[0:0]

In [191]:
#df_search_rm = df_search_rm.drop_duplicates()
df_search_rm

,id,prices_pcm,prices_pw,location,furnish_type
0,property-126878960,3467,800,Hammersmith and Fulham,furnished
1,property-127341623,4575,1056,Hammersmith and Fulham,furnished
2,property-127341047,6500,1500,Hammersmith and Fulham,furnished
3,property-127340480,4333,1000,Hammersmith and Fulham,furnished
4,property-84462349,1842,425,Hammersmith and Fulham,furnished
...,...,...,...,...,...
500,property-123293252,4875,1125,Hammersmith and Fulham,furnished
501,property-49234905,4182,965,Hammersmith and Fulham,furnished
502,property-49234758,3120,720,Hammersmith and Fulham,furnished
503,property-49234605,2383,550,Hammersmith and Fulham,furnished


In [82]:
rightmove_dict = {
    'id': get_ids_rm(bs),
    #'location': locations_dict[location],
    'prices_pcm': get_prices_rm(bs),
    'prices_pw': get_sec_prices_rm(bs)
}

In [88]:
df_search_rm = pd.DataFrame()

In [86]:
df_page_rm = pd.DataFrame(data=rightmove_dict) # dataframe with 24 rows of objects (=1 page of results)
df_page_rm['location'] = list(locations_dict.keys())[0]
df_page_rm['furnish_type'] = furnishTypes_lst[0]

# append obove dataframe to main df
df_search_rm = pd.concat([df_search_rm, df_page_rm], axis=0)
# df_search_rm = df_search_rm.drop_duplicates()

# if dataframe needs to be deleted:
# df_search_rm = df_search_rm.iloc[0:0]

In [126]:
df_search_rm['id'].nunique()

476

In [134]:
df_search_rm.head

""


### append platform column

In [ ]:
# append column with platform-name
df['platform'] = 'rightmove'

 ## NOT WORKING

In [311]:
b    beds = bs.find_all(class_ = 'l-searchResult is-list')
    beds_lst = [id.get('id') for id in ids]
    # _lst = [(str(x)) for x in ids_lst]

25

In [364]:
beds = bs.find_all(class_ = 'property-information')
beds_lst = [x.get('aria-hidden') for x in beds]
# _lst = [(str(x)) for x in ids_lst]
beds_lst

[]

In [19]:
today = dt.datetime.today().strftime('%Y-%m-%d %H:%M') # to set the date in the csv filename
df.to_csv('spotahome_{}.csv'.format(today), sep='\t')